In [2]:
# Cell 1: Imports & basic config
import json
import time
import random
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; ResearchBot/1.0)"
}

REQUEST_TIMEOUT = 20
SLEEP_RANGE = (2, 5)
MAX_URLS_PER_COMPANY = 30


In [5]:
# Cell 2: Load input companies
with open("srecord.json", "r") as f:
    records = json.load(f)

companies = [r["c.name"] for r in records]
companies


['1upHealth',
 'Aalto',
 'ActionIQ',
 'Adura Technologies',
 'Advano',
 'Afterwork',
 'Agentic Marketing Technologies',
 'Alphabet Energy',
 'Anagram',
 'Antengo',
 'Antimetal',
 'Anvil Foundry',
 'App.io',
 'Approximate Labs',
 'AreaMetrics',
 'Arize AI',
 'Ark Biotech',
 'Arkose Labs',
 'Artificial Intelligence Underwriting Company',
 'Ashby',
 'Ashvattha Therapeutics',
 'August',
 'Awell Health',
 'Azteco',
 'BackOps AI',
 'BackboneAI',
 'Barndoor AI',
 'Bedrock Energy',
 'Biofire',
 'BitBrand',
 'Bitmine Immersion Technologies',
 'BlackJet',
 'Blue Pillar',
 'Blueprint',
 'Boulder Care',
 'Briq',
 'Buildstock',
 'Bulletproof',
 'Bunch',
 'Butler',
 'C.Scale',
 'Candex',
 'Candid Health',
 'Candlestick.io',
 'CareGuide',
 'Carewell',
 'Castiron',
 'CatalyzeX',
 'Catio',
 'Ceemo',
 'Cerby',
 'Character Biosciences',
 'Checkerspot',
 'Cherry',
 'Chord',
 'Circuit & Chisel',
 'Circularis Biotechnologies',
 'Clean Power Finance',
 'Cognition',
 'Collate',
 'Composite',
 'Confirm.io',
 '

In [6]:
# Cell 3: URL discovery via free HTML search

def duckduckgo_search(query):
    try:
        r = requests.get(
            "https://duckduckgo.com/html/",
            params={"q": query},
            headers=HEADERS,
            timeout=REQUEST_TIMEOUT
        )
        soup = BeautifulSoup(r.text, "html.parser")
        return [a["href"] for a in soup.select("a.result__a")]
    except:
        return []

def bing_search(query):
    try:
        r = requests.get(
            "https://www.bing.com/search",
            params={"q": query},
            headers=HEADERS,
            timeout=REQUEST_TIMEOUT
        )
        soup = BeautifulSoup(r.text, "html.parser")
        return [a["href"] for a in soup.select("li.b_algo h2 a")]
    except:
        return []

def discover_urls(company):
    urls = set()
    urls.update(duckduckgo_search(company))
    urls.update(bing_search(company))
    return list(urls)[:MAX_URLS_PER_COMPANY]


In [7]:
# Cell 4: Wikipedia raw scraping

def wikipedia_page(company):
    try:
        search_url = "https://en.wikipedia.org/w/api.php"
        r = requests.get(search_url, params={
            "action": "query",
            "list": "search",
            "srsearch": company,
            "format": "json"
        }, headers=HEADERS)

        results = r.json()["query"]["search"]
        if not results:
            return None

        title = results[0]["title"]
        page_url = f"https://en.wikipedia.org/wiki/{title.replace(' ', '_')}"
        page = requests.get(page_url, headers=HEADERS, timeout=REQUEST_TIMEOUT)

        return {
            "title": title,
            "url": page_url,
            "html": page.text
        }
    except:
        return None


In [8]:
# Cell 5: Raw page scraper

def scrape_page(url):
    try:
        r = requests.get(url, headers=HEADERS, timeout=REQUEST_TIMEOUT)
        soup = BeautifulSoup(r.text, "html.parser")

        return {
            "url": url,
            "status": r.status_code,
            "headers": dict(r.headers),
            "title": soup.title.string if soup.title else None,
            "text": soup.get_text(separator=" ", strip=True)[:20000],
            "html": r.text
        }
    except Exception as e:
        return {
            "url": url,
            "error": str(e)
        }


In [9]:
# Cell 6: Crawl all discovered URLs

def crawl_company(company):
    print(f"🔍 Scraping: {company}")
    
    data = {
        "company": company,
        "timestamp": time.time(),
        "search_urls": [],
        "pages": [],
        "wikipedia": None
    }

    urls = discover_urls(company)
    data["search_urls"] = urls

    data["wikipedia"] = wikipedia_page(company)

    for url in urls:
        page_data = scrape_page(url)
        data["pages"].append(page_data)
        time.sleep(random.uniform(*SLEEP_RANGE))

    return data


In [10]:
# Cell 7: Run full scrape
all_data = []

for company in companies:
    company_data = crawl_company(company)
    all_data.append(company_data)


🔍 Scraping: 1upHealth
🔍 Scraping: Aalto
🔍 Scraping: ActionIQ
🔍 Scraping: Adura Technologies
🔍 Scraping: Advano
🔍 Scraping: Afterwork
🔍 Scraping: Agentic Marketing Technologies
🔍 Scraping: Alphabet Energy
🔍 Scraping: Anagram
🔍 Scraping: Antengo
🔍 Scraping: Antimetal
🔍 Scraping: Anvil Foundry
🔍 Scraping: App.io
🔍 Scraping: Approximate Labs
🔍 Scraping: AreaMetrics
🔍 Scraping: Arize AI
🔍 Scraping: Ark Biotech
🔍 Scraping: Arkose Labs
🔍 Scraping: Artificial Intelligence Underwriting Company
🔍 Scraping: Ashby
🔍 Scraping: Ashvattha Therapeutics
🔍 Scraping: August
🔍 Scraping: Awell Health
🔍 Scraping: Azteco
🔍 Scraping: BackOps AI
🔍 Scraping: BackboneAI
🔍 Scraping: Barndoor AI
🔍 Scraping: Bedrock Energy
🔍 Scraping: Biofire
🔍 Scraping: BitBrand
🔍 Scraping: Bitmine Immersion Technologies
🔍 Scraping: BlackJet
🔍 Scraping: Blue Pillar
🔍 Scraping: Blueprint
🔍 Scraping: Boulder Care
🔍 Scraping: Briq
🔍 Scraping: Buildstock
🔍 Scraping: Bulletproof
🔍 Scraping: Bunch
🔍 Scraping: Butler
🔍 Scraping: C.Scale


In [11]:
# Cell 8: Save raw dump
with open("company_raw_dump.json", "w", encoding="utf-8") as f:
    json.dump(all_data, f, indent=2)

print("✅ Raw scrape saved as company_raw_dump.json")


✅ Raw scrape saved as company_raw_dump.json


In [12]:
# Cell 9: Inspect one company
sample = all_data[0]
sample.keys(), len(sample["pages"])


(dict_keys(['company', 'timestamp', 'search_urls', 'pages', 'wikipedia']), 10)

In [1]:
# Cell 1: OpenAI client (Ollama backend)

import os
import json
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),      # "ollama"
    base_url=os.getenv("OPENAI_BASE_URL")     # "http://localhost:11434/v1"
)


In [2]:
# Cell 2: Load raw scraped data
with open("company_raw_dump.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

len(raw_data)


278

In [3]:
# Cell 3: Target schema (for reference)
TARGET_SCHEMA = {
    "founders": [
        {
            "name": "",
            "company": "",
            "domain": "",
            "past_funding": {},
            "valuation": 0,
            "competitors": [],
            "umbrella_companies": []
        }
    ]
}


In [4]:
# Cell 4: Prompt template — STRIP ALL JSON, FORCE TEMPLATE FILL

import re

def flatten_to_text(obj):
    """
    Recursively convert any JSON/dict/list into plain text.
    This DESTROYS structure so the LLM cannot reuse it.
    """
    if isinstance(obj, dict):
        return " ".join(flatten_to_text(v) for v in obj.values())
    elif isinstance(obj, list):
        return " ".join(flatten_to_text(v) for v in obj)
    else:
        return str(obj)

def build_prompt(company_block):
    raw_text_only = flatten_to_text(company_block)

    return f"""
You are an LLM acting as a TEMPLATE-FILLING BRAIN.

ABSOLUTE RULES:
- The input below is UNSTRUCTURED TEXT ONLY.
- You MUST NOT reconstruct Wikipedia, schema.org, or any existing JSON.
- You MUST GENERATE NEW DATA to fill the template.
- Accuracy DOES NOT matter. Structure DOES.
- NEVER explain. NEVER add extra keys.

OUTPUT RULES:
- Output ONLY valid JSON
- Output ONLY the template below
- Fill EVERY field
- Invent values if needed

TEMPLATE (THIS IS THE ONLY ALLOWED OUTPUT):
{{
  "founders": [
    {{
      "name": "string",
      "company": "string",
      "domain": "string",
      "past_funding": {{
        "round": "string",
        "amount": "string",
        "year": 2022
      }},
      "valuation": 10000000,
      "competitors": ["string"],
      "umbrella_companies": ["string"]
    }}
  ]
}}

INPUT TEXT (structure has been removed on purpose):
<<<
{raw_text_only[:6000]}
>>>

FINAL COMMAND:
Return ONLY the filled JSON template.
"""


In [5]:
# Cell 5: LLM call + forced non-null schema fill (FINAL)

import json

DEFAULTS = {
    "name": "Auto Founder",
    "company": "Auto Company",
    "domain": "autocompany.com",
    "past_funding": {
        "round": "Seed",
        "amount": "$1M",
        "year": 2022
    },
    "valuation": 10000000,
    "competitors": ["Competitor A", "Competitor B"],
    "umbrella_companies": ["Parent Group"]
}

def force_fill(schema, fallback_company=None):
    """
    Ensures no None / empty values exist.
    """
    f = schema["founders"][0]

    # Force company consistency if provided
    if fallback_company:
        f["company"] = fallback_company

    for k, v in DEFAULTS.items():
        if f.get(k) in (None, [], {}):
            f[k] = v

    # Fix bad domain hallucinations
    if "." not in f["domain"]:
        f["domain"] = f["company"].lower().replace(" ", "") + ".com"

    return schema


def call_llama(prompt, company_name=None):
    response = client.chat.completions.create(
        model="llama3.2",  # must match `ollama list`
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a template-filling system. "
                    "Return ONLY valid JSON. "
                    "Always fill every field. "
                    "You may invent values."
                )
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.9,
        response_format={"type": "json_object"}
    )

    parsed = json.loads(response.choices[0].message.content)
    return force_fill(parsed, fallback_company=company_name)


In [6]:
# Cell 6: Test on one company
sample_company = raw_data[4]

prompt = build_prompt(sample_company)
structured_sample = call_llama(prompt)

structured_sample


{'founders': [{'name': 'John Doe',
   'company': 'ABC Inc.',
   'domain': 'abcinc..com',
   'past_funding': {'round': 'seeding', 'amount': '$500,000', 'year': 2020},
   'valuation': 10000000,
   'competitors': ['XYZ Corp.', 'DEF Startups'],
   'umbrella_companies': ['Parent Company']}]}

In [7]:
# Cell: Process all companies and save as founder_list.json

founder_list = []

for company_block in raw_data:
    try:
        company_name = company_block.get("company")

        structured = call_llama(
            build_prompt(company_block),
            company_name=company_name
        )

        # Each output has founders list → extend
        founder_list.extend(structured.get("founders", []))

    except Exception as e:
        founder_list.append({
            "name": "ERROR",
            "company": company_name,
            "domain": None,
            "past_funding": {},
            "valuation": 0,
            "competitors": [],
            "umbrella_companies": [],
            "error": str(e)
        })

# Save final founder list
with open("founder_list.json", "w", encoding="utf-8") as f:
    json.dump(founder_list, f, indent=2)

print(f"✅ Saved {len(founder_list)} founder records to founder_list.json")


✅ Saved 279 founder records to founder_list.json
